<a href="https://colab.research.google.com/github/acollin19/comp551_A1/blob/main/assignment1_group_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

## Process:
### Dataset 1: Hepatitis
* Task 1 : Preprocessing
    1. Preprocessing
    2. Distributions
* Task 2 : Implemenmting KNN and DT
    1. KNN
    2. DT
* Task 3 : Running Experiments
    1. Compare accuracy
    2. Different K values
    3. Max tree depth
    4. Different distance/costs
    5. Plots
    6. Key features in DT
    7. Key features in KNN

### Dataset 2: Messidor features
* Task 1 : Preprocessing
    1. Preprocessing
    2. Distributions
* Task 2 : Implemenmting KNN and DT
    1. KNN
    2. DT
* Task 3 : Running Experiments
    1. Compare accuracy
    2. Different K values
    3. Max tree depth
    4. Different distance/costs
    5. Plots
    6. Key features in DT
    7. Key features in KNN


In [358]:
import pandas as pd
import numpy as np
import math
from math import sqrt
import scipy.spatial
from collections import Counter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1)

In [359]:
# Arff file to csv from https://github.com/mfahadzafar/Convert-Arff-to-CSV/blob/master/arffToCsv.py
def getCSVFromArff(fileName):
    with open(fileName + '.arff', 'r') as fin:
        data = fin.read().splitlines(True)
        i = 0
    cols = []
    for line in data:
        line = line.lower()
        if ('@data' in line):
            i+= 1
            break
        else:
            #print line
            i+= 1
            if (line.startswith('@attribute')):
                if('{' in line):
                    cols.append(line[11:line.index('{')-1])
                else:
                    cols.append(line[11:line.index(' ', 11)])
    headers = ",".join(cols)
    with open(fileName + '.csv', 'w') as fout:
        fout.write(headers)
        fout.write('\n')
        fout.writelines(data[i:])

getCSVFromArff("messidor_features") #outputs a csv file into dir

In [360]:
# Loading/cleaning datasets
hepatitis = pd.read_csv("hepatitis.data", header=None)
hepatitis = hepatitis[(hepatitis != '?').all(axis=1)]
hepatitis = hepatitis.dropna().reset_index(drop=True)
hepatitis = hepatitis.apply(pd.to_numeric)

### Hepatitis Attribute Information:

1. Class: DIE, LIVE 
2. AGE: 10, 20, 30, 40, 50, 60, 70, 80 
3. SEX: male, female 
4. STEROID: no, yes 
5. ANTIVIRALS: no, yes 
6. FATIGUE: no, yes 
7. MALAISE: no, yes 
8. ANOREXIA: no, yes 
9. LIVER BIG: no, yes 
10. LIVER FIRM: no, yes 
11. SPLEEN PALPABLE: no, yes 
12. SPIDERS: no, yes 
13. ASCITES: no, yes 
14. VARICES: no, yes 
15. BILIRUBIN: 0.39, 0.80, 1.20, 2.00, 3.00, 4.00  
16. ALK PHOSPHATE: 33, 80, 120, 160, 200, 250 
17. SGOT: 13, 100, 200, 300, 400, 500, 
18. ALBUMIN: 2.1, 3.0, 3.8, 4.5, 5.0, 6.0 
19. PROTIME: 10, 20, 30, 40, 50, 60, 70, 80, 90 
20. HISTOLOGY: no, yes 



## Task 1.2 : Distributions

In [361]:
# Hepatitis Dataset
X = hepatitis.iloc[:, hepatitis.columns != 0].values # x -> features (1-19)
y = hepatitis.iloc[:, 0].values # y -> class label (0)

In [362]:
# Split Training and Testing without sklearn
split = 0.8
n_train = math.floor(split * X.shape[0])
n_test = math.ceil(1-split * X.shape[0])
X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]
print("Total Number of rows in train:",X_train.shape[0])
print("Total Number of rows in test:",X_test.shape[0])

Total Number of rows in train: 64
Total Number of rows in test: 16


In [350]:
# Plotting


## Task 2 : K-Nearest Neighbour

Finding KNN models without using pre-existing libraries.

In [363]:
# Constructor class to initialize model parameters
class KNN:
    def __init__(self, k):
        self.k = k

    # Fit funcition takes input X and y
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    # Euclidean Distance function
    def distance(self, X1, X2):
        #distance = scipy.spatial.distance.euclidean(X1, X2)
        distance = lambda x1, x2: np.sqrt(np.sum((x1 - x2)**2, axis=-1))

    
    # Predict function takes input X and outputs y
    def predict(self, X_test):
        output = []
        for i in range(len(X_test)):
            d = []
            votes = []
            for j in range(len(X_train)):
                dist = scipy.spatial.distance.euclidean(X_train[j] , X_test[i])
                d.append([dist, j])
            d.sort()
            d = d[0:self.k]
            for d, j in d:
                votes.append(y_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            output.append(ans)
            
        return output
    
    # Evaluating prediction accuracy on true and target labels
    def evaluate_acc(self, X_test, y_test, y_train,X_train ):
        y_train_pred = self.predict(X_train)
        y_test_pred = self.predict(X_test)
        acc_test = np.sum(y_test_pred==y_test)/len(y_test)
        acc_train = np.sum(y_train_pred==y_train)/len(y_train)

        return (acc_test,acc_train)

In [364]:
classifier = KNN(5)
classifier.fit(X_train, y_train)

# Evaluating prediction accuracy on training data
test,train = classifier.evaluate_acc(X_test,y_test,y_train, X_train)

print(f'Training Accuracy: {train}')
print(f'Testing Accuracy: {test}')


Training Accuracy: 0.890625
Testing Accuracy: 0.625


## Task 2.2 : Decision Trees

# Dataset 2 : Messidor Features

## Task 1.1 Preprocessing

In [353]:
features = pd.read_csv('messidor_features.csv')
features = features[(features != '?').all(axis=1)]
features = features.dropna().reset_index(drop=True)
features = features.apply(pd.to_numeric)
features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,class
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


### Features Attribute Information:

0. The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.
1. The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack.  
* (2-7). The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5,...,1, respectively.     
* (8-15). contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the number of lesions with the diameter of the ROI to compensate different image sizes.   
16. The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patientâ€™s condition. This feature is also normalized with the diameter of the ROI.   
17. The diameter of the optic disc.   
18. The binary result of the AM/FM-based classification.   
19. Class label. 1 = contains signs of DR (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of DR. 

## Task 1.2 Distributions

In [354]:
# Messidor features dataset
X = features.iloc[:,:-1].values # x -> features (0-18)
y = features.iloc[:,-1:].values # y -> class label (19)

# Split Training and Testing without sklearn
split = 0.8
n_train = math.floor(split * X.shape[0])
n_test = math.ceil(1-split * X.shape[0])
X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]
print("Total Number of rows in train:",X_train.shape[0])
print("Total Number of rows in test:",X_test.shape[0])

Total Number of rows in train: 920
Total Number of rows in test: 231


In [355]:
# Messidor Features Plotting


## Task 2 : K-Nearest Neighbour

In [357]:
classifier = KNN(5)
classifier.fit(X_train, y_train)

# Evaluating prediction accuracy on training data
test,train = classifier.evaluate_acc(X_test,y_test,y_train, X_train)

print(f'Training Accuracy: {train}')
print(f'Testing Accuracy: {test}')


TypeError: ignored

## Task 2.2 : Decision Trees